## Installing fastText

### The first step of this tutorial is to install and build fastText.

In [2]:
%config Application.log_level='DEBUG'
import logging
logging.getLogger('fasttext').setLevel(logging.DEBUG)

In [3]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip
!cd fastText-0.9.2 && make && pip install .

--2021-10-02 15:00:31--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Résolution de github.com (github.com)… 140.82.121.3
Connexion à github.com (github.com)|140.82.121.3|:443… connecté.
requête HTTP transmise, en attente de la réponse… 302 Found
Emplacement : https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2 [suivant]
--2021-10-02 15:00:31--  https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2
Résolution de codeload.github.com (codeload.github.com)… 140.82.121.10
Connexion à codeload.github.com (codeload.github.com)|140.82.121.10|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : non indiqué [application/zip]
Enregistre : ‘v0.9.2.zip’

v0.9.2.zip              [  <=>               ]   4,17M  12,8MB/s    ds 0,3s    

2021-10-02 15:00:31 (12,8 MB/s) - ‘v0.9.2.zip’ enregistré [4369852]

Archive:  v0.9.2.zip
5b5943c118b0ec5fb9cd8d20587de2b2d3966dfe
   creating: fastText-0.9.2/
   creating: fastText-0.9.2/.circlec

  inflating: fastText-0.9.2/website/static/docs/en/html/jquery.js  
  inflating: fastText-0.9.2/website/static/docs/en/html/main_8cc.html  
  inflating: fastText-0.9.2/website/static/docs/en/html/main_8cc.js  
  inflating: fastText-0.9.2/website/static/docs/en/html/matrix_8cc.html  
  inflating: fastText-0.9.2/website/static/docs/en/html/matrix_8h.html  
  inflating: fastText-0.9.2/website/static/docs/en/html/matrix_8h_source.html  
  inflating: fastText-0.9.2/website/static/docs/en/html/menu.js  
  inflating: fastText-0.9.2/website/static/docs/en/html/menudata.js  
  inflating: fastText-0.9.2/website/static/docs/en/html/model_8cc.html  
  inflating: fastText-0.9.2/website/static/docs/en/html/model_8h.html  
  inflating: fastText-0.9.2/website/static/docs/en/html/model_8h.js  
  inflating: fastText-0.9.2/website/static/docs/en/html/model_8h_source.html  
  inflating: fastText-0.9.2/website/static/docs/en/html/namespacefasttext.html  
  inflating: fastText-0.9.2/website/static/docs/en/h

  inflating: fastText-0.9.2/website/static/img/authors/tomas_mikolov.jpg  
   creating: fastText-0.9.2/website/static/img/blog/
  inflating: fastText-0.9.2/website/static/img/blog/2016-08-18-blog-post-img1.png  
  inflating: fastText-0.9.2/website/static/img/blog/2016-08-18-blog-post-img2.png  
  inflating: fastText-0.9.2/website/static/img/blog/2017-05-02-blog-post-img1.jpg  
  inflating: fastText-0.9.2/website/static/img/blog/2017-05-02-blog-post-img2.jpg  
  inflating: fastText-0.9.2/website/static/img/blog/2017-10-02-blog-post-img1.png  
  inflating: fastText-0.9.2/website/static/img/cbo_vs_skipgram.png  
  inflating: fastText-0.9.2/website/static/img/fasttext-icon-api.png  
  inflating: fastText-0.9.2/website/static/img/fasttext-icon-bg-web.png  
  inflating: fastText-0.9.2/website/static/img/fasttext-icon-color-square.png  
  inflating: fastText-0.9.2/website/static/img/fasttext-icon-color-web.png  
  inflating: fastText-0.9.2/website/static/img/fasttext-icon-faq.png  
  inflatin

In [4]:
import fasttext

### Download directly with command line the english dataset

In [5]:
!cd fastText-0.9.2 && ./download_model.py en

 (100.00%) [==================================================>]>                                            ]======>                                       ]===============================>              ]=======================================>      ]========================================>     ]===========================================>  ]============================> ]


### Import HuggingFace

In [6]:
pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Loading the dataset "IMDB"

In [7]:
from datasets import load_dataset

In [8]:
train_dataset = load_dataset('imdb', split='train')
test_dataset = load_dataset('imdb', split='test')

In [9]:
print(train_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [10]:
print(type(train_dataset['text']))

<class 'list'>


In [11]:
print(train_dataset[0])

{'text': 'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!', 'label': 1}


### Preprocessing the dataset

In [12]:
!pip install contractions
!pip install unicodedata
!pip install beautifulsoup4
!pip install re

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [13]:
import contractions
from bs4 import BeautifulSoup
import unicodedata
import numpy as np
import re

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text):
    return contractions.fix(text)

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

def pre_process_document(document):
    # strip HTML
    document = strip_html_tags(document)
    # lower case
    document = document.lower()
    # remove extra newlines (often might be present in really noisy text)
    document = document.translate(document.maketrans("\n\t\r", "   "))
    # remove accented characters
    document = remove_accented_chars(document)
    # expand contractions    
    document = expand_contractions(document)  
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)}])')
    document = special_char_pattern.sub(" \\1 ", document)
    document = remove_special_characters(document, remove_digits=True)  
    # remove extra whitespace
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document


pre_process_corpus = np.vectorize(pre_process_document)

In [14]:
x_train_preprocessed = pre_process_corpus(train_dataset['text'])
y_train = train_dataset['label']

### Let's train the model using fasttext.

In [15]:
f = open("imdb_training.txt", "w") 
for i in range(train_dataset.num_rows):
    if y_train[i] == 1:
        f.write("__label__positive" + " " + pre_process_document(train_dataset[i]['text']) + '\n')
    if y_train[i] == 0:
        f.write("__label__negative" + " " + pre_process_document(train_dataset[i]['text']) + '\n')

In [16]:
f = open("imdb_testing.txt", "w") 
for i in range(train_dataset.num_rows):
    if y_train[i] == 1:
        f.write("__label__positive" + " " + pre_process_document(test_dataset[i]['text']) + '\n')
    if y_train[i] == 0:
        f.write("__label__negative" + " " + pre_process_document(test_dataset[i]['text']) + '\n')

In [17]:
model = fasttext.train_supervised(input="imdb_training.txt")

In [18]:
model.test("imdb_testing.txt")

(24999, 0.8731949277971118, 0.8731949277971118)

### We can do better right ?

In [19]:
model = fasttext.train_supervised(input="imdb_training.txt", autotuneValidationFile='imdb_testing.txt')

In [20]:
model.test("imdb_testing.txt")

(24999, 0.8957158286331454, 0.8957158286331454)